# OLS on a System of Regressions


This notebook illustrates how we estimate a system of regressions with OLS - and test (coefficients) across the regressions. 

## Loading Packages

In [1]:
using Dates, DelimitedFiles, Statistics, LinearAlgebra

include("jlFiles/printmat.jl")
include("jlFiles/NWFn.jl")

NWFn

## Loading Data

In [2]:
x = readdlm("Data/FFmFactorsPs.csv",',',skipstart=1)

                #yearmonth, market, small minus big, high minus low
(ym,Rme,RSMB,RHML) = (x[:,1],x[:,2]/100,x[:,3]/100,x[:,4]/100) 
x = nothing                   

T = size(Rme,1)

388

# A Function for Joint Estimation of Several Regressions (OLS)


Consider the linear regression

$
y_{it}=\beta_i^{\prime}x_{t}+\varepsilon_{it}, 
$

where $i=1,2,..,n$ indicates $n$ different dependent variables. The regressors are the *same* across the regressions. 

The next cell defines a function for this estimation. The subsequent cell tests if the intercepts (alphas) are the same across regressions.

In [3]:
"""
    OlsSureFn(y,x,m=0)

LS of y on x; for one n dependent variables, same regressors

# Usage
(b,res,yhat,Covb,R2a) = OlsSureFn(y,x,m)

# Input
- `y::Array`:     Txn, the n dependent variables
- `x::Array`:     Txk matrix of regressors (including deterministic ones)
- `m::Int`:       scalar, bandwidth in Newey-West  

# Output
- `b::Array`:     n*kx1, regression coefficients
- `u::Array`:     Txn, residuals y - yhat
- `yhat::Array`:  Txn, fitted values x*b
- `V::Array`:     matrix, covariance matrix of vec(b)
- `R2a::Number`:  n vector, R2 value

"""
function OlsSureFn(y,x,m=0)
    (T,n) = (size(y,1),size(y,2))
    k     = size(x,2)
    b     = x\y
    yhat  = x*b
    u     = y - yhat   
    g     = zeros(T,n*k)
    for i = 1:n                      
      vv      = (1+(i-1)*k):(i*k)   #1:k,(1+k):2k,...
      g[:,vv] = x.*u[:,i]           #moment conditions for y[:,i] regression
    end    
    S0    = NWFn(g,m)            #Newey-West covariance matrix
    Sxxi  = -x'x/T 
    Sxx_1 = kron(Matrix(1.0I,n,n),inv(Sxxi))
    V     = Sxx_1 * S0 * Sxx_1/T
    R2a   = 1.0 .- var(u,dims=1)./var(y,dims=1)   
    return b,u,yhat,V,R2a
end

OlsSureFn

In [4]:
println("regressing [RSMB RHML] on Rme: regression i is in column i")
(b,u,yhat,V,R2a) = OlsSureFn([RSMB RHML],[ones(T) Rme],1)
println("Regression Coeffs")
printmat(b)
println("std")
printmat(sqrt.(reshape(diag(V),2,2)))


R = [1 0 -1 0]                               #Testing if the alphas are the same
Γ = R*V*R'
test_stat = (R*vec(b))'inv(Γ)*(R*vec(b))
println("test-statictic of alpha1=alpha2 and 10% critical value of chi-square(1)")
printmat([test_stat 2.71])

regressing [RSMB RHML] on Rme: regression i is in column i
Regression Coeffs
     0.001     0.005
     0.164    -0.230

std
     0.001     0.002
     0.034     0.051

test-statictic of alpha1=alpha2 and 10% critical value of chi-square(1)
     2.112     2.710

